<a href="https://www.kaggle.com/code/search4/web-scraping-indian-states?scriptVersionId=141262633" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from bs4 import BeautifulSoup as bs
import requests
import re

#### Extract States name and url

In [2]:
state_list = []
response = requests.get('https://en.wikipedia.org/wiki/States_and_union_territories_of_India')
soup = bs(response.text)
s = soup.find('table', class_="wikitable")
for i in s.find_all('th',scope="row"):
    states = i.find('a')['title']
    link = 'https://en.wikipedia.org'+i.find('a')['href']
    state_list.append([states,link])   
    
d = soup.find('div', class_="excerpt").find_all('th', scope='row')
for i in d:
    ut = i.find('a')['title']
    ut_link = 'https://en.wikipedia.org'+i.find('a')['href']
    state_list.append([ut,ut_link])
    
state_df = pd.DataFrame(state_list,columns=['States','url'])


In [3]:
state_df.head()

,States,url
0,Andhra Pradesh,https://en.wikipedia.org/wiki/Andhra_Pradesh
1,Arunachal Pradesh,https://en.wikipedia.org/wiki/Arunachal_Pradesh
2,Assam,https://en.wikipedia.org/wiki/Assam
3,Bihar,https://en.wikipedia.org/wiki/Bihar
4,Chhattisgarh,https://en.wikipedia.org/wiki/Chhattisgarh


#### Extract all information related to each state

In [4]:
l=[]
for i in state_df['url']:
    respo = requests.get(i)
    soup = bs(respo.text)
    s = soup.find('table',class_='infobox').find('div').text
    d = soup.find_all('th',class_='infobox-label')
    e = soup.find_all('td',class_='infobox-data')

    for j,k in (zip(d,e)):  
        l.append([j.text,k.text])

#### Extracting useful information from the list 

In [6]:
cap = ['Capital','Capitaland largest city','Capitals']
form = ['Formation','Union territory','Formation(as a state)','Formation(by bifurcation)','Admission to union','As National Capital Territory']
new1 = []
totals_km2 = []
gdp=[] #US$ billion
lang = []
literacy_list = []
hdi_list = []

for i in l:
    if i[0] in form:               # formation data
        new1.append(i[1].split('[')[0])
        
    elif i[0] in cap:              # capitals
        new1.append(i[1].split('[')[0])
        
    elif i[0]=='\xa0•\xa0Total' or i[0]=='\xa0•\xa0City, Union territory':      #area
        totals_km2.append(i[1].split(' '))
        
    elif str(i[0]).startswith('\xa0•\xa0Total ('):                #gdp
        a = str(i[1]).replace(' ','',1).split('(')[-1]
        s = a.split('b')[0].replace('equivalent to ₹12\xa0trillion or US$150','US$150').split('$')[-1].split('D')[-1]
        gdp.append(s)
        
    elif i[0] in '\xa0•\xa0Official':       # language
        b = i[1].split('[')[0]
        if len(b)>0:
            lang.append(b)
            
    elif str(i[0]).startswith('HDI'):   # hdi
        hdi=str(str(i[1]).replace(' ','',1).split(' ')[0]).split('(')[0]
        hdi_list.append(str(re.split(r'[A-Z]',str(hdi))[0]).split('[')[0])
        
    elif str(i[0]).startswith('Literacy'):     # literacy rate
        literacy_list.append(str(i[1]).replace(' ','',1).split(' ')[0].split('[')[0].split('(')[0].split('%')[0])



new1.insert(-9,'New Delhi')
new1.pop(10)
new1.pop(48)  
new1.pop(70)

for i in [np.nan,np.nan,np.nan,np.nan,np.nan]:
    gdp.insert(-1,i)
    
for i in [np.nan,np.nan,np.nan]:
    hdi_list.insert(-2,i)
    
literacy_list.insert(-2,np.nan)
literacy_list.insert(-2,np.nan)
literacy_list.insert(-5,np.nan)

In [7]:
formation=[]
capitals = []
area = []
polulation = []
for i in range(len(new1)):
        if i %2==0:
            formation.append(new1[i])
        else:
            capitals.append(new1[i])
            

for i in range(len(totals_km2)):
    if i%2==0:
        area.append(totals_km2[i][0].split('\xa0k')[0])
    else:
        if len(totals_km2[i])>1:
            polulation.append(totals_km2[i][1])
        else:
            polulation.append(totals_km2[i][0])
            

            
state_df['formation'] = formation
state_df['Capital'] = capitals
state_df['Official Language']=lang
state_df['Area_in_km2'] = area
state_df['Polulation'] = polulation
state_df['GDP_USD_billion']=gdp
state_df['HDI']= hdi_list
state_df['Literacy_%']= literacy_list

In [8]:
state_df.head()

,States,url,formation,Capital,Official Language,Area_in_km2,Polulation,GDP_USD_billion,HDI,Literacy_%
0,Andhra Pradesh,https://en.wikipedia.org/wiki/Andhra_Pradesh,1 November 1956,Amaravati,"Telugu, Urdu","162,970","49,577,103",150,0.649,67.02
1,Arunachal Pradesh,https://en.wikipedia.org/wiki/Arunachal_Pradesh,20 February 1987,Itanagar,English,"83,743","1,382,611",36,0.661,65.38
2,Assam,https://en.wikipedia.org/wiki/Assam,26 January 1950,Guwahati,Assamese • Boro,"78,438","31,169,272",47,0.614,72.19
3,Bihar,https://en.wikipedia.org/wiki/Bihar,22 March 1912,Patna,Hindi,"94,163","104,099,452",99,0.581,68.15
4,Chhattisgarh,https://en.wikipedia.org/wiki/Chhattisgarh,1 November 2000,Raipur,Hindi,"135,192","29,436,231",57,0.613,70.28


In [9]:
df = state_df.copy()
df.drop('url',axis=1,inplace=True)
df

,States,formation,Capital,Official Language,Area_in_km2,Polulation,GDP_USD_billion,HDI,Literacy_%
0,Andhra Pradesh,1 November 1956,Amaravati,"Telugu, Urdu","162,970","49,577,103",150,0.649,67.02
1,Arunachal Pradesh,20 February 1987,Itanagar,English,"83,743","1,382,611",36,0.661,65.38
2,Assam,26 January 1950,Guwahati,Assamese • Boro,"78,438","31,169,272",47,0.614,72.19
3,Bihar,22 March 1912,Patna,Hindi,"94,163","104,099,452",99,0.581,68.15
4,Chhattisgarh,1 November 2000,Raipur,Hindi,"135,192","29,436,231",57,0.613,70.28
5,Goa,30 May 1987,Panaji,Konkani,"3,702","1,458,545",12,0.806,88.70
6,Gujarat,1 May 1960,Gandhinagar,"Gujarati, Hindi","196,024","60,439,692",210,0.672,78.03
7,Haryana,1 November 1966,Chandigarh,Hindi,"44,212","25,351,462",96,0.708,75.55
8,Himachal Pradesh,1 November 1956,Shimla Dharamshala (winter),Hindi,"55,673","6,864,602",20,0.725,86.06
9,Jharkhand,15 November 2000,Ranchi,Hindi,"79,716","32,988,134",53,0.598,66.41
